### Importación modulos a utilizar

In [1]:
import pandas as pd
import os
import random
from itertools import chain
from string import punctuation
from collections import Counter

### Definición de funciones necesarias

Funcion "elijo_palabras": usada para elegir 3 palabras aleatorias de los overviews de las peliculas

In [2]:
def elijo_palabras(secuencia):
    """Retorna una lista de hasta 3 items elegidos al azar de una lista"""
    if len(secuencia)<=2:
        return secuencia
    else:
        return random.sample(secuencia,3)

Revice una lista de palabras y descarte aquellas que no estan en el top_x

In [64]:
def not_top_x(top_x,lista:list[str]):
    """ Retorna lista de str que no se encuentran en top_x
    """
    return list(filter(lambda pal: pal not in top_x and (pal !='' and pal !='—'),set(lista)))

### Lectura del csv de películas

Toma del archivo csv solo las columnas a utilizar (Genre, Original_Language, Release_Date, Vote_Average, Overview, Title), en el orden especificado.
- usecols: lista de las columnas a tomar

In [49]:
path_csv=os.path.join(os.getcwd(),'..','csv','mymoviedb.csv')

columnas=['Genre','Original_Language','Release_Date','Vote_Average','Overview','Title']

peliculas=pd.read_csv(path_csv,usecols=columnas)

### Procesamiento del archivo

Reordenamiento de las columnas requeridas al siguiente orden: Genre, Original_Language, Release_Date, Vote_Average, Overview, Title

In [50]:
peliculas=peliculas.reindex(columns=columnas)

Descarte las películas que no tienen “overview” y las peliculas cuyo idioma original tenga más de 2 caracteres

In [51]:
df=peliculas[(peliculas.Overview != '')&(peliculas.Original_Language.apply(len)<3)] 

Toma todos los overviews de las peliculas y los agrupa en una lista.
Separa por palabras a cada overview, éstas en minuscula y sin caracteres especiales.

In [66]:
palabras_overviews=df.Overview.apply(lambda x: x.lower().split())

for i,lista in enumerate(palabras_overviews):
    palabras_overviews.iloc[i]=list(map(lambda x:x.strip(punctuation),lista))

Obtengo las 100 palabras mas repetidas entre todos los overviews

In [67]:
palabras=list(chain.from_iterable(palabras_overviews))
top_100 = list(zip(*Counter(palabras).most_common(100)))[0]

Toma la lista de palabras de cada overview, tomando al menos 3 palabras al azar que no se encuentran en el top 100

In [ ]:
for i,lista in enumerate(palabras_overviews):
    palabras_overviews.iloc[i]=';'.join(elijo_palabras(not_top_x(top_100,lista)))

En la columna Overview del DataFrame se encuentran ahora, las palabras elegidas

In [69]:
peliculas.Overview=palabras_overviews

Conversión a archivo csv

In [70]:
archivo_nuevo=os.path.join(os.getcwd(),'..','csv','data_set_peliculas2.csv')
peliculas.to_csv(archivo_nuevo,index=False)